In [38]:
import pandas as pd
from citipy import citipy
import numpy as np
import csv
from datetime import datetime
import json
from pandas.io.json import json_normalize

#import data
weather = pd.read_json("../ow.json") #1/1/2013-1/1/2020 hourly weather data
crime = pd.read_csv("../crime.csv")

In [39]:
drop_list = ['GEO_X', 'GEO_Y', 'DISTRICT_ID', 
             'REPORTED_DATE', 'LAST_OCCURRENCE_DATE',
            'INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE_EXTENSION']
crime.drop(columns=drop_list, inplace=True)
crime[['Date','Hour', 'am/pm']] = crime.FIRST_OCCURRENCE_DATE.str.split(expand=True)

In [40]:
crime = crime.rename(columns = {'OFFENSE_CODE':'Code', 
                               'OFFENSE_TYPE_ID':'Offense_Type', 
                               'INCIDENT_ADDRESS':'Address', 
                               'GEO_LON':'Lon',
                               'GEO_LAT':'Lat', 
                               'PRECINCT_ID':'Precint', 
                               'NEIGHBORHOOD_ID':'Neighborhood'})

In [41]:
#expand list from columns into new databases
weather_temp = (json_normalize(weather['main']))
weather_wind = json_normalize(weather['wind'])
weather_clouds = json_normalize(weather['clouds'])

In [42]:
#create new columns for desired fields from expanded list
weather_temp.head()
weather['Temp'] = weather_temp['temp']
weather['Temp Max'] = weather_temp['temp_max']
weather['Wind'] = weather_wind.speed
weather['Clouds'] = weather_clouds
#split date iso into columns, then delete unwanted columns 
weather[['Date','Hour', 'Trash', 'Trash2']] = weather.dt_iso.str.split(expand=True)
columns = ['wind', 'clouds', 'weather','main', 'timezone', 'snow', 'rain', 'dt', 'Trash', 'Trash2', 'dt_iso']
weather.drop(columns=columns, inplace=True)

In [43]:
#create dataframe for speends over 40mph 
wind_df = weather.loc[weather.Wind > 40]
wind_df

,city_name,lat,lon,Temp,Temp Max,Wind,Clouds,Date,Hour
5162,Denver,39.739236,-104.984862,60.98,64.00,47.20,90,2013-08-04,02:00:00
10599,Denver,39.739236,-104.984862,33.76,37.00,40.26,90,2014-03-18,14:00:00
11611,Denver,39.739236,-104.984862,49.96,51.80,43.84,75,2014-04-29,18:00:00
46395,Denver,39.739236,-104.984862,52.65,54.14,48.32,75,2018-04-17,22:00:00
46397,Denver,39.739236,-104.984862,46.90,48.20,40.26,20,2018-04-18,00:00:00
54312,Denver,39.739236,-104.984862,30.47,33.01,46.08,90,2019-03-13,18:00:00
59070,Denver,39.739236,-104.984862,64.54,68.00,40.26,75,2019-09-28,00:00:00
59358,Denver,39.739236,-104.984862,63.72,72.00,50.78,75,2019-10-10,00:00:00


In [44]:
#create dataframe for temps under 10f 
cold_df = weather.loc[weather['Temp Max'] < 10]
cold_df_new = cold_df.set_index('Date')
